<a href="https://colab.research.google.com/github/tmskss/Flan-T5-K8S-QA/blob/main/flan_t5_k8s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is used to fine-tune flan-t5-base model on k8s question-command pairs gathered from k8s documentation

## Installing base requirements

In [ ]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

In [2]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

## Loading the dataset

In [ ]:
dataset = load_dataset("TODO: dataset")

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
def preprocess_function(examples):
    """tokenize the text, and set the labels"""

    inputs = [prefix + doc for doc in examples['question']]
    inputs = tokenizer(inputs, max_length=256, truncation=True, padding='longest', return_tensors='pt')

    labels = tokenizer(text_target=examples['code'], max_length=256, truncation=True, padding='longest', return_tensors='pt')

    return {
        'input_ids': inputs.input_ids,
        'attention_mask': inputs.attention_mask,
        'labels': labels.input_ids,
    }


In [ ]:
# Map the preprocessing function across our dataset
tokenized_dataset_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_dataset_eval = dataset['test'].map(preprocess_function, batched=True)

## Define metrics with ROUGE

In [ ]:
# Set up Rouge score for evaluation
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

## Set up training args and start training

In [ ]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-base-k8s-question-code",
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    num_train_epochs=5,
    push_to_hub=True
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_eval,
    data_collator=None,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
from transformers import AutoModelForSeq2SeqLM

eval_model = AutoModelForSeq2SeqLM.from_pretrained("tmskss/flan-t5-base-k8s-question-code")

In [ ]:
# Define your text input
input_text = ""

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to('cuda')
eval_model.to('cuda')

# Generate text based on the input
output_ids = eval_model.generate(input_ids, max_length=100, num_return_sequences=1, num_beams=8)

# Decode the generated output
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)
